In [1]:
!pip install pywikihow
!pip install transformers
! pip install git+https://github.com/PrithivirajDamodaran/Parrot.git

     |████████████████████████████████| 47 kB 1.6 MB/s 
     |████████████████████████████████| 127 kB 9.5 MB/s 
  Created wheel for pywikihow: filename=pywikihow-0.5.7-py3-none-any.whl size=3658 sha256=55513282d16cbfbec8cf3b35b59664078c482c09e97974b8be17069a6ef5c41c
  Stored in directory: /root/.cache/pip/wheels/2b/5e/b6/3b57c2176b8efe148e8c59fcd5881e4effd67765a7e8a76cdd
Successfully built pywikihow
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 36.3 MB/s 
     |████████████████████████████████| 895 kB 51.8 MB/

In [2]:
import pandas as pd
from pywikihow import RandomHowTo, HowTo
from transformers import pipeline

#Importar Parrot (paraphraser)

from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

#Resume
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
#WikiHow
how_to = HowTo("https://www.wikihow.com/Study")

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [3]:
#Generar una semilla para la recreación de las frases
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

#Cargar el modelo parrot
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [4]:
#Read data
#post = pd.read_excel("WikiHow.xlsx", index_col=0)

In [5]:
#Create Dataset
#ERASE THE PREVIOUS DATASET
data = {"Title" : ["Titulo prueba"], "Summary": ["Summary prueba"], "Summary2": ["Summary 2 prueba"], "Summary3" : ["Summary 3 prueba"]}
post = pd.DataFrame(data)
post.to_excel("SummaryParaphraser.xlsx")

In [8]:
#Get resumes and paraphrases
for step in how_to.steps:
  dat = step.as_dict()
  resume = summarizer(dat["description"], max_length=40, min_length=5, do_sample=False)
  para_phrases = parrot.augment(input_phrase=resume[0]["summary_text"], max_return_phrases = 2, do_diverse=True)
  para1 = ""
  para2 = ""
  n = 0
  for para_phrase in para_phrases:
   if n == 0:
     para1 = para_phrase
     n += 1
   else:
     para2 = para_phrase
  prueba = {"Title":dat["summary"], "Summary": resume[0]["summary_text"], "Summary2": para1, "Summary3": para2}
  post = post.append(prueba, ignore_index=True)
  # print(dat)

In [9]:
#SAVE
post.to_excel("SummaryParaphraser.xlsx")